In [566]:
import pandas as pd
import numpy as np

In [626]:
df = pd.read_csv(r'26.02.23.csv')
bounds = pd.read_csv(r'bounds.csv')
print(df)
print(bounds)

   iteration   KeV  center  width  score
0        1.0  40.0    50.0  380.0    4.2
1        2.0  50.0   100.0  250.0    3.4
2        3.0  60.0   290.0  570.0    1.3
3        4.0  70.0   150.0  350.0    2.1
  bound    KeV    center     width
0     U  100.0  999999.0  999999.0
1     L   30.0       0.0       0.0


In [627]:
def bounds_check(vertex, bounds): # (numpy.ndarray, pd.df) 
    upper = bounds.loc[ bounds['bound'] == 'U'].drop(columns='bound').to_numpy().flatten()
    lower = bounds.loc[ bounds['bound'] == 'L'].drop(columns='bound').to_numpy().flatten()
    
    inside = np.logical_and.reduce(vertex>lower) and np.logical_and.reduce(vertex<upper)
    return inside

def simplex_step(df,bounds,scale=1.):
    Np = len(df.columns)-2 # number of parameters
    Ns = Np + 1          # dim of simplex
    iter_no = len(df)    # df.tail(1)['iteration'].iloc[0]
    
    simplex = df.tail(Ns).sort_values('score')
    worst = simplex.head(1).to_numpy().flatten()[1:(Np+1)]
    centroid = simplex.tail(Ns-1).to_numpy()[:,1:(Np+1)].mean(axis=0)
        
    new_vertex = centroid + scale*(centroid - worst)
    
    i = 1
    if not isinstance(bounds,type(None)):
        while not bounds_check(new_vertex, bounds):
            new_vertex = centroid + ( (0.5)**i  )*scale*(centroid - worst)
            i += 1
            print(new_vertex)
    
    df = pd.concat( [df[:-Ns] , simplex] )
    
    df.loc[iter_no] = [iter_no+1] + new_vertex.tolist()+[-99999.]
    return df 

In [628]:
df=simplex_step(df,bounds)
print(df)

[ 50.   5. 205.]
   iteration   KeV  center  width    score
2        3.0  60.0   290.0  570.0      1.3
3        4.0  70.0   150.0  350.0      2.1
1        2.0  50.0   100.0  250.0      3.4
0        1.0  40.0    50.0  380.0      4.2
4        5.0  50.0     5.0  205.0 -99999.0


In [629]:
new_score = 4.11111111

In [630]:
if new_score > previous_scores[-1]: # expansion rule 1
    scale = 2.0 
    #df = pd.concat( [ df.tail(1) , df.head( len(df)-1 ) ] ) 
    df = df.head( len(df)-1 )
    df = simplex_step(df,bounds,scale)
elif new_score > previous_scores[-2]: # accept simplex rule 2
    scale = 1.0
elif new_score > previous_scores[0]: # mini-expansion rule 3a
    scale = 0.5
    df = df.head( len(df)-1 )
    df = simplex_step(df,bounds,scale)
else: # mini-contraction rule 3b
    scale = -0.5
    df = df.head( len(df)-1 )
    df = simplex_step(df,bounds,scale)
    
df.at[ df.index[-1],'score'] = new_score
    
    
print(df)
print(scale)

   iteration   KeV  center  width     score
2        3.0  60.0   290.0  570.0  1.300000
3        4.0  70.0   150.0  350.0  2.100000
1        2.0  50.0   100.0  250.0  3.400000
0        1.0  40.0    50.0  380.0  4.200000
4        5.0  50.0     5.0  205.0  4.111111
1.0


In [631]:
df =simplex_step(df, bounds)
df

[ 35.    2.5 242.5]


,iteration,KeV,center,width,score
2,3.0,60.0,290.0,570.0,1.300000
3,4.0,70.0,150.0,350.0,2.100000
1,2.0,50.0,100.0,250.0,3.400000
4,5.0,50.0,5.0,205.0,4.111111
0,1.0,40.0,50.0,380.0,4.200000
5,6.0,35.0,2.5,242.5,-99999.000000


In [617]:
new_score=5.55555